<a href="https://colab.research.google.com/github/pnjha/NLP-Assignments/blob/master/nlp_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install tensorflow-gpu==2.0.0-alpha0

     |████████████████████████████████| 332.1MB 24kB/s 
     |████████████████████████████████| 419kB 62.8MB/s 
     |████████████████████████████████| 3.0MB 47.2MB/s 


In [1]:
import unicodedata
import pandas as pd
import numpy as np
import tensorflow as tf

import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import copy
import math

import keras as keras
from keras.optimizers import SGD
from keras.initializers import RandomUniform
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Embedding, Dense, LSTM, Input, Embedding, TimeDistributed, Flatten, Dropout, Activation, dot, concatenate, Bidirectional
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

AttributeError: ignored

In [2]:
tf.__version__

'2.0.0-alpha0'

In [146]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive'
/content/drive/My Drive


In [147]:
X = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.en', names=['src'])
Y_in = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.hi', names=['dest_in'])
Y_out = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.hi', names=['dest_out'])
lines = pd.concat([X[:1000],Y_in[:1000],Y_out[:1000]], axis=1)
print(len(lines))
# lines = shuffle(lines)
lines.sample(10)

1000


,src,dest_in,dest_out
190,Chicken pox is the most contagious a few days ...,फुन्सियाँ उभरने से कुछ दिनों पहले तक और सारे घ...,फुन्सियाँ उभरने से कुछ दिनों पहले तक और सारे घ...
859,The disease of women are related more to their...,"महिलाओं के रोग अधिकतर उन के उदर , स्तन , गर्भा...","महिलाओं के रोग अधिकतर उन के उदर , स्तन , गर्भा..."
210,Use full sleeve shirts and socks .,पूरी आस्तीन की कमीज तथा मोजे आदि का प्रयोग करें ।,पूरी आस्तीन की कमीज तथा मोजे आदि का प्रयोग करें ।
357,Incubation period of leprosy is three years on...,कुष्ठ रोग का संक्रमण काल ( इनक्यूवेशन पीरियड )...,कुष्ठ रोग का संक्रमण काल ( इनक्यूवेशन पीरियड )...
53,"Cancer erupts by our wrong practices , thought...","कैंसर हमारे गलत अचार , विचार व्यवहार और आहार स...","कैंसर हमारे गलत अचार , विचार व्यवहार और आहार स..."
989,There is always a dilemma for heart patients a...,हृदय रोगियों को हमेशा यह उलझन रहती है कि वे कि...,हृदय रोगियों को हमेशा यह उलझन रहती है कि वे कि...
834,With cancer infested skin healthy skin is also...,कैंसरग्रस्त त्वचा के साथ ही स्वस्थ त्वचा भी का...,कैंसरग्रस्त त्वचा के साथ ही स्वस्थ त्वचा भी का...
300,The combined vaccine is made for children and ...,संयुक्त टीका बच्चों और वयस्कों के लिए बना है ।,संयुक्त टीका बच्चों और वयस्कों के लिए बना है ।
773,There is always a need to protect eyes from th...,सूर्य की अलट्रावायलट किरणों से आँखों को हमेशा ...,सूर्य की अलट्रावायलट किरणों से आँखों को हमेशा ...
182,We will find out what is the number of cancer ...,कैंसर रजिस्ट्रर में हमें पता लग जायेगा कि राज्...,कैंसर रजिस्ट्रर में हमें पता लग जायेगा कि राज्...


In [0]:
def process_data(data,append_char):
    data = data.apply(lambda x: x.lower())
    data = data.apply(lambda x: x.strip())
    data = data.apply(lambda x: re.sub("'", '', x))
    exclude = set(string.punctuation) # Set of all special characters
    data = data.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
    remove_digits = str.maketrans('', '', digits)
    data = data.apply(lambda x: x.translate(remove_digits))
    if append_char == 1:
        data = data.apply(lambda x : '<sos> '+ x)
    elif append_char == 2:
        data = data.apply(lambda x : x + ' <eos>')
    return data

In [149]:
lines.src = process_data(lines.src,0)
lines.dest_in = process_data(lines.dest_in,1)
lines.dest_out = process_data(lines.dest_out,2)
lines.src.sample(10),lines.dest_in.sample(10),lines.dest_out.sample(10)

(355    the name of this bacteria is microbacterium le...
 527    in comparison to any other disease women die t...
 557    arbovirus is a gram positive singleformula rna...
 691          the patient of dysentery should eat barley 
 974    reiki is a heavenly power which fills the hand...
 751    the use of infrared thermography can be done i...
 521    because it is the youths who die the most of i...
 945    keep in mind that positive thought by pacifyin...
 379    yes  the hands and feet of the disabled person...
 285    paediatric dose  one dose is of   ml for child...
 Name: src, dtype: object,
 131    <sos> मासिक धर्म बंद हो जाने पर भी महिलाओं को ...
 168    <sos> अधिक मिर्च  तला  भूना  मांस  घी  मदिरा म...
 740    <sos> साथ ही शतप्रतिशत मरीज को रोगमुक्त भी किय...
 587    <sos> जब सालमोनेला टिफी युक्त कोई भी खाद्य पदा...
 421                  <sos> किसी कार्य मे ध्यान न लगाना ।
 102    <sos> इसी कारण शल्य चिकित्सक आमतौर पर गर्भाशयग...
 877    <sos> जब नवजात शिशु अपनी माँ के स्तन 

In [0]:
X, y_in, y_out = lines.src.values, lines.dest_in.values, lines.dest_out.values 
X_train, X_test, y_in_train, y_in_test, y_out_train, y_out_test = train_test_split(X, y_in, y_out, test_size = 0.2,shuffle=False)

In [151]:
src_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
src_tokenizer.fit_on_texts(X_train)

data_src = src_tokenizer.texts_to_sequences(X_train)
data_src = tf.keras.preprocessing.sequence.pad_sequences(data_src,padding='post')

print(src_tokenizer.word_index)
print(data_src)
print(len(data_src[0]))

{'the': 1, 'of': 2, 'in': 3, 'and': 4, 'is': 5, 'to': 6, 'a': 7, 'or': 8, 'be': 9, 'can': 10, 'disease': 11, 'with': 12, 'this': 13, 'are': 14, 'for': 15, 'cancer': 16, 'it': 17, 'on': 18, 'that': 19, 'not': 20, 'from': 21, 'at': 22, 'by': 23, 'also': 24, 'if': 25, 'women': 26, 'which': 27, 'has': 28, 'due': 29, 'treatment': 30, 'through': 31, 'patient': 32, 'more': 33, 'symptoms': 34, 'as': 35, 'health': 36, 'lrb': 37, 'rrb': 38, 'should': 39, 'any': 40, 'fever': 41, 'then': 42, 'body': 43, 'there': 44, 'out': 45, 'percent': 46, 'after': 47, 'been': 48, 'pain': 49, 'eyes': 50, 'most': 51, 'test': 52, 'an': 53, 'children': 54, 'tuberculosis': 55, 'blood': 56, 'patients': 57, 'done': 58, 'one': 59, 'have': 60, 'state': 61, 'diseases': 62, 'other': 63, 'than': 64, 'during': 65, 'may': 66, 'about': 67, 'found': 68, 'age': 69, 'rate': 70, 'years': 71, 'time': 72, 'its': 73, 'occurs': 74, 'very': 75, 'virus': 76, 'but': 77, 'get': 78, 'such': 79, 'some': 80, 'being': 81, 'under': 82, 'sprea

In [152]:
dest_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')

dest_tokenizer.fit_on_texts(y_in_train)
dest_tokenizer.fit_on_texts(y_out_train)

data_dest_in = dest_tokenizer.texts_to_sequences(y_in_train)
data_dest_in = tf.keras.preprocessing.sequence.pad_sequences(data_dest_in,padding='post')

data_dest_out = dest_tokenizer.texts_to_sequences(y_out_train)
data_dest_out = tf.keras.preprocessing.sequence.pad_sequences(data_dest_out,padding='post')

print(data_dest_in.shape)
print(data_dest_out.shape)

(800, 58)
(800, 58)


In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, lstm_size):
        super(Encoder, self).__init__()
        self.lstm_size = lstm_size
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.lstm_1 = tf.keras.layers.LSTM(lstm_size, return_sequences=True, return_state=True)
        self.lstm_2 = tf.keras.layers.LSTM(lstm_size, return_sequences=True, return_state=True)

    def call(self, sequence, states):
        embed = self.embedding(sequence)
        output, state_h, state_c = self.lstm_1(embed, initial_state=states)
        output, state_h, state_c = self.lstm_2(output, initial_state=[state_h,state_c])
        return output, state_h, state_c

    def init_states(self, batch_size):
        return (tf.zeros([batch_size, self.lstm_size]),
                tf.zeros([batch_size, self.lstm_size]))

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, lstm_size):
        super(Decoder, self).__init__()
        self.lstm_size = lstm_size
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.lstm = tf.keras.layers.LSTM(lstm_size, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, sequence, state):
        embed = self.embedding(sequence)
        lstm_out, state_h, state_c = self.lstm(embed, state)
        logits = self.dense(lstm_out)

        return logits, state_h, state_c

In [0]:
BATCH_SIZE = 5
EMBEDDING_SIZE = 256
LSTM_SIZE = 256

In [196]:
dataset = tf.data.Dataset.from_tensor_slices((data_src, data_dest_in, data_dest_out))
dataset = dataset.shuffle(20).batch(BATCH_SIZE)
print(dataset)

<BatchDataset shapes: ((None, 57), (None, 58), (None, 58)), types: (tf.int32, tf.int32, tf.int32)>


In [0]:
def get_model():
  src_vocab_size = len(src_tokenizer.word_index) + 1
  dest_vocab_size = len(dest_tokenizer.word_index) + 1

  encoder = Encoder(src_vocab_size, EMBEDDING_SIZE, LSTM_SIZE)
  decoder = Decoder(dest_vocab_size, EMBEDDING_SIZE, LSTM_SIZE)
  return encoder, decoder

# encoder, decoder = get_model()
# initial_states = encoder.init_states(1)
# encoder_outputs = encoder(tf.constant([[1, 2, 3]]), initial_states)
# decoder_outputs = decoder(tf.constant([[1, 2, 3]]), encoder_outputs[1:])

In [0]:
def loss_func(targets, logits):
    crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    mask = tf.math.logical_not(tf.math.equal(targets, 0))
    mask = tf.cast(mask, dtype=tf.int64)
    loss = crossentropy(targets, logits, sample_weight=mask)

    return loss

In [0]:
optimizer = tf.keras.optimizers.RMSprop()#.Adam()

In [0]:
@tf.function
def train_step(source_seq, target_seq_in, target_seq_out, en_initial_states):
    with tf.GradientTape() as tape:
        en_outputs = encoder(source_seq, en_initial_states)
        en_states = en_outputs[1:]
        de_states = en_states

        de_outputs = decoder(target_seq_in, de_states)
        logits = de_outputs[0]
        loss = loss_func(target_seq_out, logits)

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss

In [0]:
def predict(test_source_text):
    
    test_source_seq = src_tokenizer.texts_to_sequences([test_source_text])

    en_initial_states = encoder.init_states(1)
    en_outputs = encoder(tf.constant(test_source_seq), en_initial_states)

    de_input = tf.constant([[dest_tokenizer.word_index['<sos>']]])
    de_state_h, de_state_c = en_outputs[1:]
    out_words = []

    while True:
        de_output, de_state_h, de_state_c = decoder(de_input, (de_state_h, de_state_c))
        de_input = tf.argmax(de_output, -1)
        try:
          out_words.append(dest_tokenizer.index_word[de_input.numpy()[0][0]])
        except:
          out_words.append('<unk>')
        if out_words[-1] == '<eos>' or len(out_words) >= 50:
            break

    output = ' '.join(out_words)
    return output 

In [202]:
NUM_EPOCHS = 500
encoder, decoder = get_model()
for e in range(NUM_EPOCHS):
    en_initial_states = encoder.init_states(BATCH_SIZE)

    for batch, (source_seq, target_seq_in, target_seq_out) in enumerate(dataset.take(-1)):
        loss = train_step(source_seq, target_seq_in,target_seq_out, en_initial_states)

    print('Epoch {} Loss {:.8f}'.format(e + 1, loss.numpy()))

Epoch 1 Loss 1.82389593
Epoch 2 Loss 1.69894862
Epoch 3 Loss 1.63988566
Epoch 4 Loss 1.57252705
Epoch 5 Loss 1.50233638
Epoch 6 Loss 1.46893036
Epoch 7 Loss 1.42830205
Epoch 8 Loss 1.39157164
Epoch 9 Loss 1.36779773
Epoch 10 Loss 1.33917236
Epoch 11 Loss 1.32315075
Epoch 12 Loss 1.28881323
Epoch 13 Loss 1.24929988
Epoch 14 Loss 1.22956121
Epoch 15 Loss 1.17968893
Epoch 16 Loss 1.15208995
Epoch 17 Loss 1.14039886
Epoch 18 Loss 1.10294437
Epoch 19 Loss 1.12072206
Epoch 20 Loss 1.10304797
Epoch 21 Loss 1.03958702
Epoch 22 Loss 1.02000570
Epoch 23 Loss 0.97170293
Epoch 24 Loss 0.94644701
Epoch 25 Loss 0.94374043
Epoch 26 Loss 0.90076149
Epoch 27 Loss 0.95787627
Epoch 28 Loss 0.86760360
Epoch 29 Loss 0.82587928
Epoch 30 Loss 0.81926119
Epoch 31 Loss 0.80467516
Epoch 32 Loss 0.79271460
Epoch 33 Loss 0.76998103
Epoch 34 Loss 0.72117049
Epoch 35 Loss 0.76108295
Epoch 36 Loss 0.73239714
Epoch 37 Loss 0.68391353
Epoch 38 Loss 0.68065095
Epoch 39 Loss 0.64740169
Epoch 40 Loss 0.67802125
Epoch 41 

In [0]:
def calculate_bleu_score(actual_string, predicted_string):
    actual_string = copy.deepcopy(actual_string)
    predicted_string = copy.deepcopy(predicted_string)
    reference = re.split("\s",actual_string.strip())
    candidate = re.split("\s",predicted_string.strip())
    try:
      reference.remove('<eos>')
      candidate.remove('<sos>')
    except:
      pass
    smoothie = SmoothingFunction().method4
    score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
    return score

In [0]:
def get_prediction(X,y):
  for index,line in enumerate(X):
    output = predict(line)
    print("source: ",line)
    print("actual: ",y[index])
    print("predicted: ",output)
    print("BLEU Score: ",calculate_bleu_score(output,y[index]))

In [3]:
# get_prediction(X_test,y_in_test)
get_prediction(X_train,y_in_train)

NameError: ignored